# Documentação do Projeto

## 1. Pré-processamento dos CSV

Nesta etapa, foi inicialmente criado o arquivo de SQL, seguindo o passo a passo abaixo:

1. **Análise do diagrama conceitual**: A estrutura do banco de dados foi definida com base em um diagrama conceitual que orientou a criação das tabelas e relacionamentos.

2. **Criação do comando de `INSERT` em SQL**: As colunas foram transformadas de `INT` para `BIGINT`, adequando o banco de dados ao tamanho dos dados presentes nas colunas. A seguir estão os comandos SQL para a criação das tabelas:


```sql
CREATE TABLE country (
  country VARCHAR(128),
  country_id UUID PRIMARY KEY
);

CREATE TABLE state (
  state VARCHAR(128),
  country_id UUID,
  state_id UUID PRIMARY KEY,
  FOREIGN KEY (country_id) REFERENCES country(country_id)
);

CREATE TABLE city (
  city VARCHAR(256),
  state_id UUID,
  city_id INT PRIMARY KEY,
  FOREIGN KEY (state_id) REFERENCES state(state_id)
);

CREATE TABLE d_month (
  month_id BIGINT PRIMARY KEY,
  action_month BIGINT
);

CREATE TABLE d_year (
  year_id INT PRIMARY KEY,
  action_year INT
);

CREATE TABLE d_week (
  week_id BIGINT PRIMARY KEY,
  action_week BIGINT
);

CREATE TABLE d_weekday (
  weekday_id BIGINT PRIMARY KEY,
  action_weekday VARCHAR(128)
);

CREATE TABLE d_time (
  time_id BIGINT PRIMARY KEY,
  action_timestamp TIMESTAMP,
  week_id BIGINT,
  month_id BIGINT,
  year_id BIGINT,
  weekday_id BIGINT,
  FOREIGN KEY (week_id) REFERENCES d_week(week_id),
  FOREIGN KEY (month_id) REFERENCES d_month(month_id),
  FOREIGN KEY (year_id) REFERENCES d_year(year_id),
  FOREIGN KEY (weekday_id) REFERENCES d_weekday(weekday_id)
);

CREATE TABLE customers (
  customer_id UUID PRIMARY KEY,
  first_name VARCHAR(128),
  last_name VARCHAR(128),
  customer_city BIGINT,
  cpf BIGINT,
  country_name VARCHAR(128),
  FOREIGN KEY (customer_city) REFERENCES city(city_id)
);

CREATE TABLE accounts (
  account_id UUID PRIMARY KEY,
  customer_id UUID,
  created_at TIMESTAMP,
  status VARCHAR(128),
  account_branch VARCHAR(128),
  account_check_digit VARCHAR(128),
  account_number VARCHAR(128),
  FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

CREATE TABLE transfer_ins (
  id UUID PRIMARY KEY,
  account_id UUID,
  amount FLOAT,
  transaction_requested_at BIGINT,
  transaction_completed_at BIGINT,
  status VARCHAR(128),
  FOREIGN KEY (account_id) REFERENCES accounts(account_id),
  FOREIGN KEY (transaction_requested_at) REFERENCES d_time(time_id),
  FOREIGN KEY (transaction_completed_at) REFERENCES d_time(time_id)
);

CREATE TABLE transfer_outs (
  id UUID PRIMARY KEY,
  account_id UUID,
  amount FLOAT,
  transaction_requested_at BIGINT,
  transaction_completed_at BIGINT,
  status VARCHAR(128),
  FOREIGN KEY (account_id) REFERENCES accounts(account_id),
  FOREIGN KEY (transaction_requested_at) REFERENCES d_time(time_id),
  FOREIGN KEY (transaction_completed_at) REFERENCES d_time(time_id)
);

CREATE TABLE pix_movements (
  id UUID PRIMARY KEY,
  account_id UUID,
  in_or_out VARCHAR(128),
  pix_amount FLOAT,
  pix_requested_at BIGINT,
  pix_completed_at BIGINT,
  status VARCHAR(128),
  FOREIGN KEY (account_id) REFERENCES accounts(account_id),
  FOREIGN KEY (pix_requested_at) REFERENCES d_time(time_id),
  FOREIGN KEY (pix_completed_at) REFERENCES d_time(time_id)
); 
```

3. **Criação de um arquivo em Python para tratamento de dados**: Um script de ETL foi desenvolvido para realizar diversos tratamentos nas bases. 
Esse ETL está disponível em: conversao.ipynb

As etapas incluíram:
   - Conversão de colunas para o formato `UUID` para manter as ligações de chaves: `['state_id', 'country_id', 'account_id', 'customer_id', 'id']`.
   - Conversão das colunas `['pix_requested_at', 'pix_completed_at', 'transaction_requested_at', 'transaction_completed_at']` de string para `BIGINT`, conforme o diagrama.


## 2. Carga no PostgreSQL
Explique como os dados foram carregados no PostgreSQL. Inclua comandos SQL, ferramentas utilizadas ou bibliotecas em Python, se aplicável.

1. **Inserção dos dados no PostgreSQL**: Os dados foram inseridos manualmente no banco de dados PostgreSQL. Também foi considerado o desenvolvimento de um script em Python que respeitasse a sequência lógica de inserção das tabelas, dada a natureza relacional do banco de dados.

![Pix Movements](Imagens\PIX_MOVEMENTS.PNG)
![State](Imagens\STATE.PNG)
![Accounts](Imagens\ACCOUNTS.PNG)


## 3. Criação do SQL

Para a Criação do SQL, foram utilizados os seguintes scripts:

### Questão 1

```sql
WITH transitions_ins AS (
  SELECT 
    A.ACCOUNT_ID, 
    A.ACCOUNT_NUMBER || '-' || A.ACCOUNT_CHECK_DIGIT AS ACCOUNT_NUMBER, 
    TO_CHAR(C.ACTION_TIMESTAMP, 'YYYY-MM') AS TRANSITIONS_DATE, 
    ROUND(SUM(B.AMOUNT):: numeric, 2) AS AMOUNT_IN
  FROM ACCOUNTS A 
    LEFT JOIN TRANSFER_INS B ON A.ACCOUNT_ID = B.ACCOUNT_ID 
    LEFT JOIN D_TIME C ON B.transaction_completed_at = C.TIME_ID 
  WHERE B.STATUS = 'completed' 
  GROUP BY 
    A.ACCOUNT_ID, 
    A.ACCOUNT_NUMBER || '-' || A.ACCOUNT_CHECK_DIGIT, 
    TO_CHAR(C.ACTION_TIMESTAMP, 'YYYY-MM')
), 
transitions_ins_pix AS (
  SELECT 
    A.ACCOUNT_ID, 
    A.ACCOUNT_NUMBER || '-' || A.ACCOUNT_CHECK_DIGIT AS ACCOUNT_NUMBER, 
    TO_CHAR(C.ACTION_TIMESTAMP, 'YYYY-MM') AS TRANSITIONS_DATE, 
    ROUND(SUM(B.PIX_AMOUNT):: numeric, 2) AS AMOUNT_IN
  FROM ACCOUNTS A 
    LEFT JOIN PIX_MOVEMENTS B ON A.ACCOUNT_ID = B.ACCOUNT_ID 
    LEFT JOIN D_TIME C ON B.PIX_COMPLETED_AT = C.TIME_ID 
  WHERE 
    B.STATUS = 'completed' 
    AND IN_OR_OUT = 'pix_in' 
  GROUP BY 
    A.ACCOUNT_ID, 
    A.ACCOUNT_NUMBER || '-' || A.ACCOUNT_CHECK_DIGIT, 
    TO_CHAR(C.ACTION_TIMESTAMP, 'YYYY-MM')
), 
transitions_outs AS (
  SELECT 
    A.ACCOUNT_ID, 
    A.ACCOUNT_NUMBER || '-' || A.ACCOUNT_CHECK_DIGIT AS ACCOUNT_NUMBER, 
    TO_CHAR(C.ACTION_TIMESTAMP, 'YYYY-MM') AS TRANSITIONS_DATE, 
    ROUND(SUM(B.AMOUNT):: numeric, 2) AS AMOUNT_OUT
  FROM ACCOUNTS A 
    LEFT JOIN TRANSFER_OUTS B ON A.ACCOUNT_ID = B.ACCOUNT_ID 
    LEFT JOIN D_TIME C ON B.transaction_completed_at = C.TIME_ID 
  WHERE B.STATUS = 'completed' 
  GROUP BY 
    A.ACCOUNT_ID, 
    A.ACCOUNT_NUMBER || '-' || A.ACCOUNT_CHECK_DIGIT, 
    TO_CHAR(C.ACTION_TIMESTAMP, 'YYYY-MM')
), 
transitions_out_pix AS (
  SELECT 
    A.ACCOUNT_ID, 
    A.ACCOUNT_NUMBER || '-' || A.ACCOUNT_CHECK_DIGIT AS ACCOUNT_NUMBER, 
    TO_CHAR(C.ACTION_TIMESTAMP, 'YYYY-MM') AS TRANSITIONS_DATE, 
    ROUND(SUM(B.PIX_AMOUNT):: numeric, 2) AS AMOUNT_OUT
  FROM ACCOUNTS A 
    LEFT JOIN PIX_MOVEMENTS B ON A.ACCOUNT_ID = B.ACCOUNT_ID 
    LEFT JOIN D_TIME C ON B.PIX_COMPLETED_AT = C.TIME_ID 
  WHERE 
    B.STATUS = 'completed' 
    AND IN_OR_OUT = 'pix_out' 
  GROUP BY 
    A.ACCOUNT_ID, 
    A.ACCOUNT_NUMBER || '-' || A.ACCOUNT_CHECK_DIGIT, 
    TO_CHAR(C.ACTION_TIMESTAMP, 'YYYY-MM')
),

account_monthly_balance AS (
   SELECT 
      COALESCE(ins.ACCOUNT_ID, ins_pix.ACCOUNT_ID, outs.ACCOUNT_ID, outs_pix.ACCOUNT_ID) AS ACCOUNT_ID,
      COALESCE(ins.ACCOUNT_NUMBER, ins_pix.ACCOUNT_NUMBER, outs.ACCOUNT_NUMBER, outs_pix.ACCOUNT_NUMBER) AS ACCOUNT_NUMBER,
      COALESCE(ins.TRANSITIONS_DATE, ins_pix.TRANSITIONS_DATE, outs.TRANSITIONS_DATE, outs_pix.TRANSITIONS_DATE) AS TRANSITIONS_DATE,
      COALESCE(ins.AMOUNT_IN, 0) + COALESCE(ins_pix.AMOUNT_IN, 0) AS TOTAL_AMOUNT_IN,
      COALESCE(outs.AMOUNT_OUT, 0) + COALESCE(outs_pix.AMOUNT_OUT, 0) AS TOTAL_AMOUNT_OUT,
      (COALESCE(ins.AMOUNT_IN, 0) + COALESCE(ins_pix.AMOUNT_IN, 0)) - (COALESCE(outs.AMOUNT_OUT, 0) + COALESCE(outs_pix.AMOUNT_OUT, 0)) AS MONTHLY_BALANCE
   FROM transitions_ins ins
   FULL OUTER JOIN transitions_ins_pix ins_pix ON ins.ACCOUNT_ID = ins_pix.ACCOUNT_ID AND ins.TRANSITIONS_DATE = ins_pix.TRANSITIONS_DATE
   FULL OUTER JOIN transitions_outs outs ON ins.ACCOUNT_ID = outs.ACCOUNT_ID AND ins.TRANSITIONS_DATE = outs.TRANSITIONS_DATE
   FULL OUTER JOIN transitions_out_pix outs_pix ON ins.ACCOUNT_ID = outs_pix.ACCOUNT_ID AND ins.TRANSITIONS_DATE = outs_pix.TRANSITIONS_DATE
)

SELECT * FROM account_monthly_balance
ORDER BY ACCOUNT_ID, TRANSITIONS_DATE;
```


![Imagem 1](Imagens\Consulta1.PNG)


### Questão 2:

```sql
WITH account_out_by_pix AS (
  SELECT 
    A.ACCOUNT_ID, 
    A.ACCOUNT_NUMBER || '-' || A.ACCOUNT_CHECK_DIGIT AS ACCOUNT_NUMBER, 
    TO_CHAR(C.ACTION_TIMESTAMP, 'YYYY-MM') AS TRANSITIONS_DATE, 
    ROUND(SUM(B.PIX_AMOUNT):: numeric, 2) AS AMOUNT_OUT,
    COUNT(ID) AS QUANTITY
  FROM ACCOUNTS A 
    LEFT JOIN PIX_MOVEMENTS B ON A.ACCOUNT_ID = B.ACCOUNT_ID 
    LEFT JOIN D_TIME C ON B.PIX_COMPLETED_AT = C.TIME_ID 
  WHERE 
    B.STATUS = 'completed' 
    AND IN_OR_OUT = 'pix_out' 
  GROUP BY 
    A.ACCOUNT_ID, 
    A.ACCOUNT_NUMBER || '-' || A.ACCOUNT_CHECK_DIGIT, 
    TO_CHAR(C.ACTION_TIMESTAMP, 'YYYY-MM')
),

monthly_summary AS (
   SELECT 
      TRANSITIONS_DATE AS MONTH,
      COUNT(DISTINCT ACCOUNT_ID) AS NUMBER_OF_ACCOUNTS,
      SUM(QUANTITY) AS NUMBER_OF_TRANSFERS,
      ROUND(SUM(AMOUNT_OUT), 2) AS AMOUNT_OF_TRANSFERS
   FROM account_out_by_pix
   GROUP BY TRANSITIONS_DATE
),
monthly_metrics AS (
  SELECT 
    M.MONTH,
    M.NUMBER_OF_ACCOUNTS,
    M.NUMBER_OF_TRANSFERS,
    M.AMOUNT_OF_TRANSFERS,
    LAG(M.NUMBER_OF_TRANSFERS) OVER (ORDER BY M.MONTH) AS L1M_NUMBER_OF_TRANSFERS,
    AVG(M.NUMBER_OF_TRANSFERS) OVER (ORDER BY M.MONTH ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) AS L3M_AVG_NUMBER_OF_TRANSFERS,
    LAG(M.NUMBER_OF_ACCOUNTS) OVER (ORDER BY M.MONTH) AS L1M_NUMBER_OF_ACCOUNTS,
    AVG(M.NUMBER_OF_ACCOUNTS) OVER (ORDER BY M.MONTH ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) AS L3M_AVG_NUMBER_OF_ACCOUNTS
  FROM monthly_summary M
)

SELECT 
  MONTH,
  NUMBER_OF_ACCOUNTS,
  NUMBER_OF_TRANSFERS,
  AMOUNT_OF_TRANSFERS,
  ROUND((NUMBER_OF_TRANSFERS::NUMERIC / NULLIF(L1M_NUMBER_OF_TRANSFERS, 0) - 1) * 100, 3) AS "%L1M Number of Transfers",
  ROUND(((NUMBER_OF_TRANSFERS::NUMERIC / NULLIF(L3M_AVG_NUMBER_OF_TRANSFERS, 0)) - 1) * 100, 3) AS "%L3M AVG Number of Transfers",
  ROUND((NUMBER_OF_ACCOUNTS::NUMERIC / NULLIF(L1M_NUMBER_OF_ACCOUNTS, 0) - 1) * 100, 3) AS "%L1M Number of accounts",
  ROUND(((NUMBER_OF_ACCOUNTS::NUMERIC / NULLIF(L3M_AVG_NUMBER_OF_ACCOUNTS, 0)) - 1) * 100, 3) AS "%L3M AVG Number of accounts"
FROM monthly_metrics
ORDER BY MONTH;
```

![Imagem 2](Imagens\Consulta2.PNG)

## 4. Criação dos Indicadores

Para atender às necessidades da cliente ou da área de negócios, decidi dividir os indicadores em três categorias: **métricas de volume**, **métricas end-to-end** (relacionadas à aquisição e retenção de usuários), e **métricas operacionais**. Cada uma dessas categorias é detalhada a seguir:

### Métricas de Volume
Essas métricas permitem avaliar a escala e o valor financeiro das transações realizadas:

- **Total de Transações**: Quantidade total de transações realizadas.
- **Valor Total das Transações**: Somatório do valor de todas as transações.
- **Comparação Pix vs. Outros Métodos**: Percentual do volume e do valor das transações realizadas via Pix em comparação com métodos tradicionais.

### Métricas End-to-End
Essas métricas oferecem uma visão sobre a utilização do Pix, focando em aspectos de aquisição e retenção de usuários:

- **Acompanhamento de Usuários Ativos**: Percentual de usuários que utilizam o Pix regularmente.
- **Oportunidade de Conversão (Usuários Inativos)**: Quantidade de usuários que ainda não utilizaram o Pix, indicando potencial de crescimento.
- **Acompanhamento de Novos Usuários**: Percentual de crescimento de novos usuários que aderem ao Pix ao longo do tempo.
- **Acompanhamento do Volume Transferido**: Volume médio transferido por usuário ativo, para monitorar o engajamento.
- **Estados com Maior Aderência**: Identificação dos estados com maior proporção de usuários ativos no Pix.

### Métricas Operacionais
Essas métricas refletem a eficiência e confiabilidade operacional do Pix:

- **Taxa de Sucesso e Falha**: Percentual de transações Pix que foram concluídas com sucesso versus as que falharam.
- **Aceitabilidade do Pix**: Percentual de usuários que preferem o Pix em relação a métodos tradicionais, com base na taxa de sucesso e nas falhas relatadas.




## 5. Criação do Painel

O painel foi criado em Power BI, em um protótipo estruturado, mas sem tempo alocado a questões visuais de telas de fundo e animações. O painel está disponível no caminho relativo -> Painel\painel.pbix



## 6. Criação da Análise em Python

A análise em Python foi uma etapa extra, que considerei criar para obter análises da base de uma maneira mais intuitiva. Ela está disponível para leitura no caminho -> Analise python\analise_pix.ipynb


## 7. Insights

### Análises de PIX com status de completed em Python

- Volume Total de Transações
  - **Total de Transações**: 237.283
  - Nos fornece o indicativo da alta utilização do sistema de pagamento via PIX, refletindo forte adesão e confiança da base.

- Média de Transações
  - **Valor Médio**: R$ 999,61
  - Sugere que o PIX é utilizado frequentemente para transações significativas.

- Variabilidade nos Valores
  - **Desvio Padrão**: R$ 578,21
  - Alta variabilidade informa que o PIX é usado tanto para transações desde pequenos valores, quanto grandes.

- Distribuição dos Valores
  - **Valor Mínimo**: R$ 0,01
  - **Valor Máximo**: R$ 1.999,99

- **Quartis**:
  - **25%** das transações: inferiores a R$ 497,28.
  - **Mediana**: R$ 999,05, indicando que metade das transações são abaixo deste valor.
  - **75%** das transações: inferiores a R$ 1.501,84.

- Comportamento dos Usuários
  - O PIX é amplamente adotado para uma variedade de transações, desde pequenas compras diárias até pagamentos de maior valor.
  - A variabilidade sugere diferentes perfis de usuários, desde os que usam o PIX para pagamentos diários até os que realizam transações maiores, possivelmente, até para empresas, mas precisariamos de informações dos usuários que recebem o valor, não somente enviam.


### Análises de PIX com status de failed em Python
- **Solicitações PIX que falharam**: Foram registradas 12.559 solicitações de transações.
  
- **Valor Médio de Transação**: O valor médio foi de **R$ 1.004,38**, sugerindo que, além de frequente, são solicitações envolvendo valores elevados.
  
- **Análise de Quartis**:
  - **Primeiro Quartil (25%)**: Transações abaixo de **R$ 495,69**.
  - **Mediana (50%)**: Transações de até **R$ 1.008,74**, um valor alto-moderado, que pode corresponder a pagamentos de contas, como mercado ou transporte por aplicativo.
  - **Terceiro Quartil (75%)**: Transações até **R$ 1.508,73**, valor próximo do máximo registrado (**R$ 1.999,82**), sugerindo usos para transferências de valores mais altos.
  
- **Desvio Padrão**:  **R$ 582,72**, indica uma variabilidade significativa nos valores das transações, com valores extremos entre **R$ 0,01** e **R$ 1.999,82**.

- **Distribuição de solicitação de entradas e Saídas**: A relação é próxima de 50%/50%, mas as solicitações de entradas superam ligeiramente as saídas.

- **Horário das Solicitações**:
  - **61% das solicitações** ocorrem durante o horário comercial (9h às 18h), indicando que o PIX pode estar sendo utilizado como alternativa de pagamento tanto por empresas quanto por consumidores em geral.
  
- **Distribuição Geográfica**:
  - **Estados com Maior Volume de Transações falhadas**: Os cinco estados com o maior número de transações falhadas via PIX são **MG, SP, RS, SC** e **GO**.


### Análises do Painel, de forma geral

- **Taxa Geral de Aprovação**: A taxa de aprovação das solicitações de PIX está em **94,96%**, enquanto o método antigo apresenta uma taxa de **95,07%**, indicando uma paridade próxima no sucesso das solicitações de movimentação financeira.

- **Picos de Uso e Novos Usuários**:
  - **Meses com Maior Uso do PIX**: Agosto, maio e outubro foram os meses com maior número de usuários utilizando a ferramenta.
  - **Meses com Maior Adoção de Novos Usuários**: Julho, maio e setembro registraram os maiores índices de novos usuários aderindo ao modelo PIX.

- **Saldo de Entradas e Saídas via PIX nos Primeiros Meses**:
  - Nos seis primeiros meses do ano, os usuários realizaram mais saques do que depósitos via PIX. Isso sugere que a ferramenta era usada principalmente para pagamentos urgentes, quando havia pouco ou nenhum tempo para processar o método antigo ou realizar saques no caixa.
  - Se considerarmos dados reais e não apenas para um caso de estudo, essa tendência pode ser explicada pela crise de saúde pública enfrentada no ano, que aumentou a necessidade de pagamentos rápidos e emergenciais.

- **Usuários Não Adesos**: **21 pessoas** (0,55% da base de clientes) ainda não utilizaram o PIX em nenhuma transação.

- **Índices de Falhas**:
  - Os meses com maior índice de falhas no PIX foram novembro (5,18%), seguido por agosto (5,16%) e maio (5,15%). Em comparação, o método antigo registrou falhas abaixo de 5% em todos os meses.
  - Apenas em três meses (março, com 4,88%; outubro, com 4,92%; e julho, com 4,99%), o PIX apresentou menos falhas que o método antigo.



## 8. Documentação e Insights

Criação da documentação, unificando tudo realizado e visualizado durante o projeto e também documentando todas as mudanças e insights obitidos para uma visão rápida de apresentação para uma área de negócios, por exemplo.

